In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
df = pd.read_csv('/content/AAPL.csv')

FileNotFoundError: [Errno 2] File /content/AAPL.csv does not exist: '/content/AAPL.csv'

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df = df[['Date', 'Close']]
df.head()

In [ ]:
df.plot(x='Date', y='Close', figsize=(20,10))

In [ ]:
def ilinechart(df, x, y, groups=None, title=''):
    fig = px.line(df, x=x, y=y, color=groups, title=title, 
                  template='none')
    fig.show()

In [ ]:
ilinechart(df, 'Date', 'Close')

In [ ]:
df['Rolling_Mean'] = df['Close'].rolling(window=252).mean()
df['Rolling_Std'] = df['Close'].rolling(window=252).std()

In [ ]:
melted = pd.melt(df, id_vars='Date', value_vars=['Close', 'Rolling_Mean', 'Rolling_Std'], 
                 var_name='Variable', value_name='Value')

ilinechart(melted, 'Date', 'Value', groups='Variable')

In [ ]:
import statsmodels.api as sm

adf_test = sm.tsa.stattools.adfuller(df['Close'])

results = pd.Series(adf_test[0:4], 
                    index=['ADF Test Statistic','P-Value',
                           '# Lags Used','# Observations Used'])

for key, value in adf_test[4].items():
    results['Critical Value (%s)'%key] = value

print('Augmented Dickey-Fuller Test Results:\n')
print(results)

In [ ]:
# H_0: Time series is not stationary
# H_a: Time series is stationary
def check_sig(results):
  if results['P-Value'] < 0.05:
    print('P-Value: ', results['P-Value'])
    print('Reject null hypothesis')
    print('Data is stationary')
  else:
    print('P-Value: ', results['P-Value'])
    print('Fail to reject null hypothesis')
    print('Data is not stationary')

In [ ]:
check_sig(results)

In [ ]:
df.head()

In [ ]:
# df['Close'] - df['Close'].shift(1)
df['Diff_1'] = df['Close'].diff()

In [ ]:
ilinechart(df, 'Date', 'Diff_1')

In [ ]:
adf_test = sm.tsa.stattools.adfuller(df['Diff_1'][1:])

results = pd.Series(adf_test[0:4], 
                    index=['ADF Test Statistic','P-Value',
                           '# Lags Used','# Observations Used'])

for key, value in adf_test[4].items():
    results['Critical Value (%s)'%key] = value

print('Augmented Dickey-Fuller Test Results:\n')
print(results)

In [ ]:
check_sig(results)

In [ ]:
df['Diff_2'] = df['Diff_1'].diff()

In [ ]:
df['returns'] = df['Close'].pct_change()

In [ ]:
ilinechart(df, 'Date', 'returns')

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

model = ARIMA(df['Close'], order=(1,1,1)).fit()